---

# Chapter 13: Layered and Modular Architecture (N-Tier, Hexagonal, Onion)

## Opening Context

As applications grow from a handful of files to thousands of classes, the need for a coherent **architectural blueprint** becomes critical. Without deliberate structure, codebases devolve into tightly coupled monoliths where a change in one place unpredictably breaks another. Developers face a fundamental question: **How do we partition a system so that it remains understandable, testable, and adaptable over years of evolution?**

This chapter answers that question by exploring three foundational architectural patterns that govern system‑wide structure:

1. **N‑Tier Architecture** – the classic layered separation of presentation, business logic, and data.
2. **Hexagonal Architecture** (Ports and Adapters) – which places the business logic at the centre, isolated from external concerns.
3. **Onion Architecture** – a refinement of the layered idea that enforces strict dependency rules, keeping the domain model pure.

Each pattern represents a different trade‑off between simplicity, isolation, and adaptability. By understanding them, you can choose the right foundation for your project and communicate architectural decisions clearly with your team.

---

## 13.1 N-Tier Architecture

### Intent
*Separate the concerns of an application into distinct layers, each with a specific responsibility, and arrange them in a stack where higher layers depend on lower layers.*

### The Problem

Imagine a simple order‑processing system. The naive approach is to write everything in a single file or module:

```typescript
// ❌ NO LAYERING: All responsibilities mixed together
import express from 'express';
import { Client } from 'pg';

const app = express();
app.use(express.json());

app.post('/orders', async (req, res) => {
  const { customerId, items } = req.body;
  
  // 1. Validation (should be in presentation layer)
  if (!customerId || !items || items.length === 0) {
    return res.status(400).send('Invalid order');
  }
  
  // 2. Business logic: calculate total (should be in business layer)
  let total = 0;
  for (const item of items) {
    total += item.price * item.quantity;
  }
  
  // 3. Database access (should be in data layer)
  const db = new Client({ /* connection */ });
  await db.connect();
  try {
    await db.query('BEGIN');
    const orderResult = await db.query(
      'INSERT INTO orders (customer_id, total) VALUES ($1, $2) RETURNING id',
      [customerId, total]
    );
    const orderId = orderResult.rows[0].id;
    
    for (const item of items) {
      await db.query(
        'INSERT INTO order_items (order_id, product_id, quantity, price) VALUES ($1, $2, $3, $4)',
        [orderId, item.productId, item.quantity, item.price]
      );
    }
    await db.query('COMMIT');
    res.status(201).json({ orderId });
  } catch (err) {
    await db.query('ROLLBACK');
    res.status(500).send('Database error');
  } finally {
    await db.end();
  }
});

app.listen(3000);
```

**Problems**:
- **Low cohesion** – The endpoint mixes HTTP concerns, business rules, and database details.
- **High coupling** – Changing the database would require rewriting the entire endpoint.
- **Untestable** – You cannot test the business logic without spinning up a database and an HTTP server.
- **Poor maintainability** – Every new feature touches all parts of the code, increasing the risk of regression.

### The Solution: Three-Tier Architecture

**N‑Tier** (most commonly three‑tier) architecture organises the system into horizontal layers:

- **Presentation Layer** – Handles user interaction (UI, API endpoints) and translates requests to/from the business layer.
- **Business Logic Layer** (also called Domain or Service layer) – Implements the core rules and workflows.
- **Data Access Layer** – Abstracts database operations (repositories, DAOs).

The golden rule: **A layer may only depend on the layer directly beneath it.** The presentation layer knows about the business layer, and the business layer knows about the data layer, but not vice versa. This unidirectional flow reduces coupling and increases maintainability.

Let's refactor the order example into a clean three‑tier structure.

#### Data Layer (Repository)

```typescript
// data/order.repository.ts
import { db } from './database';

export interface OrderData {
  customerId: string;
  items: Array<{ productId: string; quantity: number; price: number }>;
  total: number;
}

export class OrderRepository {
  async save(order: OrderData): Promise<string> {
    const client = await db.connect();
    try {
      await client.query('BEGIN');
      const orderResult = await client.query(
        'INSERT INTO orders (customer_id, total) VALUES ($1, $2) RETURNING id',
        [order.customerId, order.total]
      );
      const orderId = orderResult.rows[0].id;
      
      for (const item of order.items) {
        await client.query(
          'INSERT INTO order_items (order_id, product_id, quantity, price) VALUES ($1, $2, $3, $4)',
          [orderId, item.productId, item.quantity, item.price]
        );
      }
      await client.query('COMMIT');
      return orderId;
    } catch (err) {
      await client.query('ROLLBACK');
      throw err;
    } finally {
      client.release();
    }
  }
}
```

**Explanation**:
- The repository is responsible **only** for persistence. It knows nothing about HTTP or business rules.
- It receives a structured `OrderData` object and returns the new order ID.

#### Business Layer (Service)

```typescript
// business/order.service.ts
import { OrderRepository } from '../data/order.repository';

export class OrderService {
  constructor(private orderRepository: OrderRepository) {}

  async createOrder(customerId: string, items: Array<{ productId: string; quantity: number; price: number }>): Promise<string> {
    // Business rule: validate order
    if (!customerId || items.length === 0) {
      throw new Error('Invalid order');
    }
    
    // Business rule: calculate total
    const total = items.reduce((sum, item) => sum + item.price * item.quantity, 0);
    
    // Delegate persistence to repository
    const orderData = { customerId, items, total };
    return this.orderRepository.save(orderData);
  }
}
```

**Explanation**:
- The service contains **business logic** (validation, calculation). It depends on the repository abstraction (injected via constructor).
- It does not know about HTTP or the database – it only uses the repository interface.

#### Presentation Layer (Controller)

```typescript
// presentation/order.controller.ts
import { Request, Response } from 'express';
import { OrderService } from '../business/order.service';

export class OrderController {
  constructor(private orderService: OrderService) {}

  async createOrder(req: Request, res: Response): Promise<void> {
    try {
      const { customerId, items } = req.body;
      const orderId = await this.orderService.createOrder(customerId, items);
      res.status(201).json({ orderId });
    } catch (error) {
      res.status(400).json({ error: error.message });
    }
  }
}
```

**Explanation**:
- The controller handles HTTP requests/responses. It parses input, calls the service, and formats the output.
- It depends only on the service, not on the repository or database.

#### Composition Root (Application entry point)

```typescript
// app.ts
import express from 'express';
import { OrderController } from './presentation/order.controller';
import { OrderService } from './business/order.service';
import { OrderRepository } from './data/order.repository';

const app = express();
app.use(express.json());

// Wire dependencies (manual DI)
const orderRepository = new OrderRepository();
const orderService = new OrderService(orderRepository);
const orderController = new OrderController(orderService);

app.post('/orders', (req, res) => orderController.createOrder(req, res));

app.listen(3000);
```

**Explanation**:
- The composition root (here, the main file) wires all dependencies together. This is where we decide which concrete implementations to use.
- This structure adheres to **Dependency Injection** (Chapter 11) – dependencies are passed in, not created inside classes.

### Benefits of N-Tier

1. **Separation of Concerns** – Each layer has a clear responsibility.
2. **Testability** – You can test the business layer in isolation by mocking the repository.
3. **Maintainability** – Changes to the database only affect the data layer; changes to the UI only affect the presentation layer.
4. **Team Scaling** – Different teams can work on different layers with well‑defined interfaces.

### Drawbacks and Limitations

- **Leaky Abstractions** – Sometimes business logic leaks into the presentation layer (e.g., formatting dates) or into the data layer (e.g., stored procedures with business rules).
- **Database‑Centric Design** – The data layer often dictates the structure of the business objects, leading to an **anemic domain model** (objects with only getters/setters, no behaviour).
- **Rigid Dependency Direction** – While layers depend downward, changes in lower layers can ripple upward. For example, a change in the database schema may force changes in the business layer (if the repository returns database‑specific types).
- **Not Suitable for Complex Domains** – For applications with rich business logic, a simple three‑tier architecture may not provide enough isolation for the domain model.

These limitations led to the development of alternative architectures that place the domain at the centre.

---

## 13.2 Hexagonal Architecture (Ports and Adapters)

### Intent
*Allow an application to be equally driven by users, programs, automated tests, or batch scripts, and to be developed and tested in isolation from its external runtime devices and databases.*
— Alistair Cockburn

**Hexagonal Architecture** (also called **Ports and Adapters**) inverts the dependency direction of traditional layering. Instead of having the business layer depend on the data layer, the business layer defines **ports** (interfaces) that are implemented by **adapters** (external integrations). The core domain becomes completely independent of technical concerns.

### Key Concepts

- **Core (Domain)** – Contains the business logic, entities, and use cases. It knows nothing about the outside world.
- **Ports** – Interfaces that define how the core interacts with the outside. There are two types:
  - **Driving Ports** (inbound) – Used by external actors (UI, API, tests) to trigger use cases. Example: `OrderService` interface.
  - **Driven Ports** (outbound) – Used by the core to reach external resources (database, message queue). Example: `OrderRepository` interface.
- **Adapters** – Implementations of ports.
  - **Driving Adapters** – e.g., REST controllers, CLI handlers, message listeners.
  - **Driven Adapters** – e.g., PostgreSQL repository, Redis cache, Kafka producer.

The core defines the ports; the adapters plug into them. This is a direct application of the **Dependency Inversion Principle** (DIP) from SOLID.

### Refactoring the Order Example to Hexagonal

#### Core Domain (no external dependencies)

```typescript
// core/entities/order.ts
export interface OrderItem {
  productId: string;
  quantity: number;
  price: number;
}

export class Order {
  constructor(
    public readonly customerId: string,
    public readonly items: OrderItem[],
    public readonly total: number
  ) {}

  static create(customerId: string, items: OrderItem[]): Order {
    if (!customerId || items.length === 0) {
      throw new Error('Invalid order');
    }
    const total = items.reduce((sum, item) => sum + item.price * item.quantity, 0);
    return new Order(customerId, items, total);
  }
}
```

**Explanation**:
- The `Order` entity encapsulates business rules (validation, total calculation) in a static factory method.
- It is a plain object with no references to databases or HTTP.

#### Ports (Interfaces)

```typescript
// core/ports/outbound/order-repository.port.ts
import { Order } from '../../entities/order';

export interface OrderRepository {
  save(order: Order): Promise<string>;
}
```

```typescript
// core/ports/inbound/order-service.port.ts
import { OrderItem } from '../../entities/order';

export interface OrderService {
  createOrder(customerId: string, items: OrderItem[]): Promise<string>;
}
```

**Explanation**:
- These interfaces belong to the core. They are the **ports** through which the outside world communicates with the core (driving) and the core communicates with the outside (driven).

#### Core Use Case Implementation

```typescript
// core/use-cases/create-order.use-case.ts
import { Order } from '../entities/order';
import { OrderRepository } from '../ports/outbound/order-repository.port';
import { OrderService } from '../ports/inbound/order-service.port';

export class CreateOrderUseCase implements OrderService {
  constructor(private orderRepository: OrderRepository) {}

  async createOrder(customerId: string, items: OrderItem[]): Promise<string> {
    const order = Order.create(customerId, items);
    return this.orderRepository.save(order);
  }
}
```

**Explanation**:
- The use case implements the **driving port** (`OrderService`).
- It depends on the **driven port** (`OrderRepository`), but only on the interface – not on a concrete implementation.
- The use case contains the **application‑specific business rules** (orchestrating the entity and repository).

#### Driven Adapter (Database)

```typescript
// infrastructure/adapters/outbound/postgres-order.repository.ts
import { Order } from '../../../core/entities/order';
import { OrderRepository } from '../../../core/ports/outbound/order-repository.port';
import { db } from '../database';

export class PostgresOrderRepository implements OrderRepository {
  async save(order: Order): Promise<string> {
    const client = await db.connect();
    try {
      await client.query('BEGIN');
      const orderResult = await client.query(
        'INSERT INTO orders (customer_id, total) VALUES ($1, $2) RETURNING id',
        [order.customerId, order.total]
      );
      const orderId = orderResult.rows[0].id;
      
      for (const item of order.items) {
        await client.query(
          'INSERT INTO order_items (order_id, product_id, quantity, price) VALUES ($1, $2, $3, $4)',
          [orderId, item.productId, item.quantity, item.price]
        );
      }
      await client.query('COMMIT');
      return orderId;
    } catch (err) {
      await client.query('ROLLBACK');
      throw err;
    } finally {
      client.release();
    }
  }
}
```

**Explanation**:
- This is a concrete implementation of the `OrderRepository` port.
- It lives in the **infrastructure** layer, which is outside the core.

#### Driving Adapter (REST Controller)

```typescript
// infrastructure/adapters/inbound/rest/order.controller.ts
import { Request, Response } from 'express';
import { OrderService } from '../../../../core/ports/inbound/order-service.port';

export class OrderController {
  constructor(private orderService: OrderService) {}

  async createOrder(req: Request, res: Response): Promise<void> {
    try {
      const { customerId, items } = req.body;
      const orderId = await this.orderService.createOrder(customerId, items);
      res.status(201).json({ orderId });
    } catch (error) {
      res.status(400).json({ error: error.message });
    }
  }
}
```

**Explanation**:
- The controller depends on the **driving port** (`OrderService`). It doesn't know about the concrete use case.
- This allows us to swap the use case implementation (e.g., for testing) without changing the controller.

#### Composition Root (Wiring)

```typescript
// main.ts
import express from 'express';
import { OrderController } from './infrastructure/adapters/inbound/rest/order.controller';
import { CreateOrderUseCase } from './core/use-cases/create-order.use-case';
import { PostgresOrderRepository } from './infrastructure/adapters/outbound/postgres-order.repository';

const app = express();
app.use(express.json());

// Instantiate adapters
const orderRepository = new PostgresOrderRepository();

// Instantiate core use case (injecting driven adapter)
const createOrderUseCase = new CreateOrderUseCase(orderRepository);

// Instantiate driving adapter (injecting core use case via port)
const orderController = new OrderController(createOrderUseCase);

app.post('/orders', (req, res) => orderController.createOrder(req, res));

app.listen(3000);
```

**Explanation**:
- The composition root creates all objects and wires them together.
- Notice that the core (`CreateOrderUseCase`) knows nothing about `PostgresOrderRepository` – it only knows the `OrderRepository` interface.
- The direction of dependencies points **inward** toward the core.

### Testing the Core in Isolation

One of the greatest benefits of Hexagonal Architecture is the ability to test the core without any infrastructure.

```typescript
// test/unit/create-order.use-case.test.ts
import { CreateOrderUseCase } from '../../core/use-cases/create-order.use-case';
import { OrderRepository } from '../../core/ports/outbound/order-repository.port';
import { Order } from '../../core/entities/order';

describe('CreateOrderUseCase', () => {
  it('should create an order and save it', async () => {
    // Arrange: create an in‑memory repository (test double)
    const mockRepo: OrderRepository = {
      save: jest.fn().mockResolvedValue('order-123')
    };
    const useCase = new CreateOrderUseCase(mockRepo);
    
    // Act
    const orderId = await useCase.createOrder('cust-1', [
      { productId: 'p1', quantity: 2, price: 10 }
    ]);
    
    // Assert
    expect(orderId).toBe('order-123');
    expect(mockRepo.save).toHaveBeenCalledWith(
      expect.objectContaining({
        customerId: 'cust-1',
        total: 20
      })
    );
  });
});
```

**Explanation**:
- We provide a mock implementation of `OrderRepository` that lives entirely in memory.
- The test runs fast and does not require a database or HTTP server.
- This is possible because the core depends only on abstractions, not concretions.

### Benefits of Hexagonal Architecture

- **Domain Isolation** – Business rules are protected from external changes (database, frameworks).
- **Testability** – Core can be tested with lightweight test doubles.
- **Flexibility** – You can swap out adapters (e.g., change from PostgreSQL to MongoDB) without touching the core.
- **Technology Independence** – The core does not depend on any framework or library, making it easier to migrate or adopt new technologies.
- **Parallel Development** – Teams can work on different adapters simultaneously, as long as the port interfaces are agreed upon.

### Trade‑offs

- **Increased Complexity** – More interfaces and indirection than a simple layered architecture.
- **Learning Curve** – Developers need to understand the concept of ports and adapters.
- **Over‑engineering for Simple CRUD** – If your application is mostly data‑driven with little business logic, Hexagonal may add unnecessary overhead.

---

## 13.3 Onion Architecture

### Intent
*Structure an application into concentric layers, with the domain model at the core, and apply the Dependency Inversion Principle so that outer layers depend on inner layers, never the other way around.*
— Jeffrey Palermo

**Onion Architecture** is very similar to Hexagonal Architecture in its goal of isolating the domain, but it emphasises **layers** rather than ports and adapters. The typical layers are:

- **Domain Model** (Core) – Entities, value objects, aggregates.
- **Domain Services** – Interfaces and implementations of domain‑specific logic that doesn't fit into an entity.
- **Application Services** (Use Cases) – Orchestrate domain objects to fulfill user stories. They depend on domain services and repository interfaces.
- **Infrastructure** (outermost) – Database, UI, external APIs, etc. This layer implements the interfaces defined by inner layers.

The key rule: **Dependencies point inward**. Outer layers can depend on inner layers, but inner layers never depend on outer layers.

### Refactoring the Order Example to Onion

#### Core (Domain Model)

```typescript
// domain/entities/order.ts
export class Order {
  constructor(
    public readonly customerId: string,
    public readonly items: OrderItem[],
    public readonly total: number
  ) {}

  static create(customerId: string, items: OrderItem[]): Order {
    if (!customerId || items.length === 0) throw new Error('Invalid order');
    const total = items.reduce((sum, i) => sum + i.price * i.quantity, 0);
    return new Order(customerId, items, total);
  }
}

export interface OrderItem {
  productId: string;
  quantity: number;
  price: number;
}
```

#### Domain Service Interfaces (Ports)

In Onion, these are often placed in the **Domain** layer or a separate **Application** layer. Here we'll define repository interfaces in the Domain layer (since they are abstractions of something the domain needs).

```typescript
// domain/repositories/order-repository.interface.ts
import { Order } from '../entities/order';

export interface OrderRepository {
  save(order: Order): Promise<string>;
}
```

#### Application Services (Use Cases)

```typescript
// application/use-cases/create-order.use-case.ts
import { Order } from '../../domain/entities/order';
import { OrderRepository } from '../../domain/repositories/order-repository.interface';

export class CreateOrderUseCase {
  constructor(private orderRepository: OrderRepository) {}

  async execute(customerId: string, items: OrderItem[]): Promise<string> {
    const order = Order.create(customerId, items);
    return this.orderRepository.save(order);
  }
}
```

**Explanation**:
- The application service coordinates the domain entity and the repository interface.
- It does not know about the concrete repository.

#### Infrastructure Layer

```typescript
// infrastructure/persistence/postgres-order.repository.ts
import { Order } from '../../domain/entities/order';
import { OrderRepository } from '../../domain/repositories/order-repository.interface';
import { db } from './database';

export class PostgresOrderRepository implements OrderRepository {
  async save(order: Order): Promise<string> {
    // ... same as before
  }
}
```

```typescript
// infrastructure/web/order.controller.ts
import { Request, Response } from 'express';
import { CreateOrderUseCase } from '../../application/use-cases/create-order.use-case';

export class OrderController {
  constructor(private createOrderUseCase: CreateOrderUseCase) {}

  async createOrder(req: Request, res: Response): Promise<void> {
    try {
      const { customerId, items } = req.body;
      const orderId = await this.createOrderUseCase.execute(customerId, items);
      res.status(201).json({ orderId });
    } catch (error) {
      res.status(400).json({ error: error.message });
    }
  }
}
```

#### Composition Root

```typescript
// main.ts
import { CreateOrderUseCase } from './application/use-cases/create-order.use-case';
import { PostgresOrderRepository } from './infrastructure/persistence/postgres-order.repository';
import { OrderController } from './infrastructure/web/order.controller';

const orderRepository = new PostgresOrderRepository();
const createOrderUseCase = new CreateOrderUseCase(orderRepository);
const orderController = new OrderController(createOrderUseCase);

// ... express setup
```

### Comparison with Hexagonal

Onion and Hexagonal are very similar in philosophy. The main differences are:

- **Hexagonal** focuses on **ports and adapters** – the core defines interfaces, and adapters plug in. It explicitly distinguishes between driving and driven ports.
- **Onion** structures the code into **layers** (Domain, Application, Infrastructure) with clear dependency direction. It's often easier to explain to developers familiar with layered architecture.

Both achieve the same goal: **isolating the domain** and **inverting dependencies**. Many practitioners consider them two flavours of the same idea, often grouped under **Clean Architecture** (Robert C. Martin), which adds a few more layers but retains the same inward‑pointing dependency rule.

---

## 13.4 Choosing the Right Architecture

| Architecture | When to Use | Strengths | Weaknesses |
|--------------|-------------|-----------|------------|
| **N‑Tier**   | Simple CRUD apps, rapid prototyping, small teams. | Simple to understand, easy to implement, good for data‑centric apps. | Leaky abstractions, domain often anemic, changes ripple. |
| **Hexagonal** | Complex business logic, long‑lived applications, need for high testability. | Complete isolation of domain, easy to test, technology independent. | More interfaces, steeper learning curve. |
| **Onion**    | Similar to Hexagonal, but when you prefer a layered mental model. | Same as Hexagonal, with clear layer separation. | Same as Hexagonal. |

For many modern enterprise applications, **Hexagonal** or **Onion** (Clean Architecture) is the recommended starting point because they protect the business logic from the inevitable changes in frameworks, databases, and external APIs.

---

## Chapter Summary

This chapter explored three architectural styles for organising large applications:

1. **N‑Tier Architecture** separates presentation, business, and data layers with unidirectional dependencies. It’s simple and effective for many applications but can lead to an anemic domain model and leaky abstractions.

2. **Hexagonal Architecture** (Ports and Adapters) places the core domain at the centre, defines ports (interfaces) for inbound and outbound communication, and lets adapters implement those ports. This inverts dependencies, making the core independent of external concerns and highly testable.

3. **Onion Architecture** is a similar concentric layering approach that enforces inward‑pointing dependencies. It provides a clear separation between domain, application, and infrastructure layers.

All three architectures aim to manage complexity through separation of concerns and controlled dependencies. The choice depends on the nature of your project, the complexity of the domain, and the team’s familiarity with these patterns.

**Key Insight**: Regardless of the architectural style, the principles of **Dependency Inversion** and **Separation of Concerns** are the true foundation. Mastering these principles allows you to adapt any architecture to your needs.

---

## Next Chapter Preview

**Chapter 14: UI and Separation Patterns (MVC, MVVM, MVP)**

We now turn our attention to patterns that separate user interface concerns from business logic. Chapter 14 will cover the classic **Model‑View‑Controller (MVC)** pattern, the **Model‑View‑ViewModel (MVVM)** pattern popular in frameworks like WPF and Vue, and the **Model‑View‑Presenter (MVP)** pattern. You’ll learn how these patterns apply the principles of separation to the presentation tier, making UI code more testable and maintainable.



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../3. modern_adaptations_functional_patterns/12. functional_programming_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='14. ui_and_separation_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
